In [130]:
%run input/Format.ipynb
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

In [131]:
iOption0 = 0
sim = 1
Min_Entries = 10
N_files = 1
Do_save = 1

In [132]:
N_hists = 6
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan,root.kTeal]
file_path="input/"
file_names="my-m_ee_Run14AuAu_85th_new_19858_1065runs" 
#m_ee_Run14AuAu_84th_new_19850_337runs m_ee_Run14AuAu_83rd_new_19842_1065runs m_ee_Run14AuAu_85th_new_19858_120runs
hist_select_3D_names = ["DCA12_hist_0","DCA12_hist_1","DCA12_hist_2","DCA12_hist_3","DCA12_hist_4","hist_dca_x","hist_dca_y"]
N_hists = len(hist_select_3D_names)

In [133]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_new_v6"#piminus_50M_v7 piminus_50M_v8 piminus_50M_v9
    N_hists = N_hists - 2

In [134]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ihist in range(N_hists):
    hists_read.append(infile.Get(hist_select_3D_names[ihist]))
    hists_read[-1].SetDirectory(root.nullptr)
infile.Close()

In [135]:
z_edges = [-12+3*i for i in range(9)]
z_means = [-10.5+3*i for i in range(8)]
z_edges = array('d', z_edges)
z_means = array('d', z_means)
phi_edges = [-1.5 + 0.30*i for i in range(21)]
phi_means = [-1.35 + 0.30*i for i in range(20)]
phi_edges = array('d', phi_edges)
phi_means = array('d', phi_means)
def get_phi_zed_bin(in_value):
    if in_value > 179: in_value -= 180
    zed = int(in_value / 20)
    phi = int((in_value - zed * 20) )
    return zed, phi
def get_phi_value(phi_bin):
    return phi_means[phi_bin] if phi_bin >= 0 and phi_bin < len(phi_means) else 0
def get_zed_value(zed_bin):
    return z_means[zed_bin] if zed_bin >= 0 and zed_bin < len(z_means) else 0

In [136]:
def copycat_hist(hist,seed=0):
    seed += root.gRandom.Integer(1000)
    new_hist = root.TH3D(hist.GetName()+f"{seed}", hist.GetTitle()+f"{seed}", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()),\
        hist.GetNbinsY(), hist.GetYaxis().GetBinLowEdge(1), hist.GetYaxis().GetBinUpEdge(hist.GetNbinsY()),\
        hist.GetNbinsZ(), hist.GetZaxis().GetBinLowEdge(1), hist.GetZaxis().GetBinUpEdge(hist.GetNbinsZ()))
    for ix in range(1, hist.GetNbinsX()+1):
        for iy in range(1, hist.GetNbinsY()+1):
            for iz in range(1, hist.GetNbinsZ()+1):
                new_hist.SetBinContent(ix, iy, iz, hist.GetBinContent(ix, iy, iz))
    return new_hist

In [137]:
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 1600, 600)
c1.Divide(2)
c1.cd(1)
hist_proj = in_hist.ProjectionX(in_hist.GetName()+"hist_proj", 21, 30, 1, 160)
hist_proj.Draw("")
c1.cd(2)
hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
hist_projy.Draw("")

c1.Draw()


In [138]:
fit_func = root.TF1("fit_func", "[0]*cos(0.3*x-1.5)+[1]*sin(0.3*x-1.5)+[2]", 0, 320)
fit_func.SetParameters(0., 0., 0.)
fit_func.FixParameter(2, 0.)

In [139]:
ihits = 1
in_hist = copycat_hist(hists_read[ihits])
for ibinx in range(in_hist.GetNbinsX()*0):
    for ibiny in range(in_hist.GetNbinsY()):
        for ibinz in range(in_hist.GetNbinsZ()):
            if ibiny<20 or ibiny>30:
                in_hist.SetBinContent(ibinx, ibiny, ibinz, 0)
hist_proj = in_hist.Project3D("xz")

c1 = root.TCanvas("c1", "c1", 800, 600)
profile_x = hist_proj.ProfileX(hist_proj.GetName()+"profileX")
profile_x.Rebin(1)
#profile_x.Fit(fit_func, "Q", "", 50, 60)
profile_x.Draw("")
c1.Draw()


In [140]:
ihits = 1
in_hist = copycat_hist(hists_read[ihits])
for ibinx in range(in_hist.GetNbinsX()*0):
    for ibiny in range(in_hist.GetNbinsY()):
        for ibinz in range(in_hist.GetNbinsZ()):
            if ibinx<20 or ibinx>30:
                in_hist.SetBinContent(ibinx, ibiny, ibinz, 0)
hist_proj = in_hist.Project3D("yz")

c1 = root.TCanvas("c1", "c1", 800, 600)
profile_y = hist_proj.ProfileX(hist_proj.GetName()+"profileX",1,51)
profile_y.Rebin(1)
profile_y.Draw("")
c1.Draw()
    

In [141]:
x_offsets = array("d")
y_offsets = array("d")
x_abscissas = array("d")
y_abscissas = array("d")
x_arrays = {}
y_arrays = {}
for ibinx in range(profile_x.GetNbinsX()):
    x_offsets.append(profile_x.GetBinContent(ibinx+1))
    x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
for ibiny in range(profile_y.GetNbinsX()):
    y_offsets.append(profile_y.GetBinContent(ibiny+1))
    y_abscissas.append(profile_y.GetBinCenter(ibiny+1))
for i, (xx, yy) in enumerate(zip(x_abscissas, x_offsets)):
    z_bin, phi_bin = get_phi_zed_bin(xx)
    phi = get_phi_value(phi_bin)
    if z_bin not in x_arrays:
        x_arrays[z_bin] = [array('d'),array('d')]
    x_arrays[z_bin][0].append((yy))
    x_arrays[z_bin][1].append((phi))
for i, (yy, xx) in enumerate(zip(y_abscissas, y_offsets)):
    z_bin, phi_bin = get_phi_zed_bin(yy)
    phi = get_phi_value(phi_bin)
    if z_bin not in y_arrays:
        y_arrays[z_bin] = [array('d'),array('d')]
    y_arrays[z_bin][0].append((xx))
    y_arrays[z_bin][1].append((phi))
print("x_arrays", x_arrays)
print("y_arrays", y_arrays)

x_arrays {0: [array('d', [0.0, 0.0, 0.0, -1.4893617021276595, 0.449438202247191, -0.30927835051546393, 1.8562874251497006, -1.0655737704918034, 10.909090909090908, -23.79310344827586, 0.0, -310.0, -250.0, -3.767123287671233, -2.7574750830564785, -0.7692307692307693, 0.37383177570093457, 3.936651583710407, 13.963963963963964, 0.0]), array('d', [-1.35, -1.05, -0.7500000000000001, -0.4500000000000002, -0.15000000000000013, 0.1499999999999999, 0.44999999999999973, 0.75, 1.0499999999999998, 1.3499999999999996, 1.65, 1.9499999999999997, 2.2499999999999996, 2.55, 2.85, 3.15, 3.4499999999999997, 3.7499999999999996, 4.049999999999999, 4.35])], 1: [array('d', [0.0, 0.0, 0.0, 2.1266968325791855, 1.1012658227848102, -0.17278617710583152, -2.608955223880597, -3.599374021909233, -1.0256410256410255, -6.425531914893617, 0.0, 0.0, -30.0, 0.6808859721082855, 1.5454545454545454, -0.4078164825828377, -1.5015576323987538, -1.6928886091881687, -11.534246575342467, -95.0]), array('d', [-1.35, -1.05, -0.7500

In [142]:
c3 = root.TCanvas("c3", "c3", 1600, 1600)
c3.Divide(2, 4)
graphs_x = []
graphs_y = []
for iz in range(len(z_means)):
    c3.cd(iz+1)
    if iz not in x_arrays: continue
    graph_x = root.TGraph(len(x_arrays[iz][0]), x_arrays[iz][1], x_arrays[iz][0])
    graph_x.SetTitle(f"Z={get_zed_value(iz)}")
    graph_x.GetXaxis().SetTitle("Phi")
    graph_x.GetYaxis().SetTitle("DCA X/Y")
    graph_x.SetMarkerStyle(20)
    graph_x.SetMarkerColor(colors[iz])
    graphs_x.append(graph_x)
    graph_y = root.TGraph(len(y_arrays[iz][0]), y_arrays[iz][1], y_arrays[iz][0])
    graph_y.SetTitle(f"Z={get_zed_value(iz)}")
    graph_y.GetXaxis().SetTitle("Phi")
    graph_y.GetYaxis().SetTitle("DCA Y")
    graph_y.SetMarkerStyle(21)
    graph_y.SetMarkerColor(colors[iz+1])
    graph_x.SetMaximum(50)
    graph_x.SetMinimum(-50)
    graph_x.Draw("AP")
    graph_y.Draw("same P")
    graphs_y.append(graph_y)
c3.Draw()

In [143]:
if True and len(x_arrays)==len(y_arrays) and len(x_abscissas)==len(y_abscissas):
    print(f"const double offsets_dca_x[{len(x_arrays)}][{len(x_arrays[0][0])}] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {", end="")
        for i in range(len(x_arrays[iz][0])):
            if i > 0: print(", ", end="")
            print(f"{x_arrays[iz][0][i]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")
    print(f"const double offsets_dca_y[{len(y_arrays)}][{len(y_arrays[0][0])}] =" + " {")
    for iz in range(len(y_arrays)):
        if iz > 0: print(",")
        print("    {", end="")
        for i in range(len(y_arrays[iz][0])):
            if i > 0: print(", ", end="")
            print(f"{y_arrays[iz][0][i]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca_x[8][20] = {
    {0, 0, 0, -0.000149, 4.49e-05, -3.09e-05, 0.000186, -0.000107, 0.00109, -0.00238, 0, -0.031, -0.025, -0.000377, -0.000276, -7.69e-05, 3.74e-05, 0.000394, 0.0014, 0},
    {0, 0, 0, 0.000213, 0.00011, -1.73e-05, -0.000261, -0.00036, -0.000103, -0.000643, 0, 0, -0.003, 6.81e-05, 0.000155, -4.08e-05, -0.00015, -0.000169, -0.00115, -0.0095},
    {0, 0, 0, -6.82e-05, -3.94e-05, 0.000169, 0.000467, 0.000445, 0.000357, -0.000427, 0, -0.025, -0.023, 0.000406, 0.0002, -1.44e-05, -0.000439, -0.000356, -0.000429, -0.00233},
    {0, 0, 0, -0.000436, -0.000228, 0.000336, 0.000689, 0.00119, 0.000984, 0.000244, 0, 0, -0.003, 0.00109, 0.000548, 4.06e-07, -0.000725, -0.000894, -0.00141, -0.003},
    {0, 0, 0, -0.00021, -1.39e-05, 0.000183, 0.000164, -0.000194, 0.000174, 0.000782, 0, 0, 0, 0.00105, 0.000395, -5.04e-06, -0.000531, -0.00108, -0.00111, -0.0042},
    {0, 0, 0, 0.000545, 7.98e-05, -5.56e-05, -0.000152, 0.000103, -0.00024, 0.000809, 0, 0, 0, 0.000948, 

In [144]:
if True:
    
    print(f"const double offsets_dca_y[{len(y_arrays)}][2] =" + " {")
    for iz in range(len(y_arrays)):
        if iz > 0: print(",")
        print("    {"+f"{(y_arrays[iz][0][4]+y_arrays[iz][0][5])/10000:.3g}"+f", {y_arrays[iz][0][15]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca_y[8][2] = {
    {0.000292, -0.00024},
    {0.000867, 0.000551},
    {-0.00153, 0.0011},
    {-0.00365, 0.00202},
    {-0.00161, 0.0018},
    {0.00091, 0.00202},
    {-0.00272, 0.00205},
    {-0.00521, 0.00202}
};


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.